In [1]:
import MA4Star_Strategy as MA4
import os
import mplfinance as mpf
import matplotlib.pyplot as plt

2025-07-18 18:33:24.843588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-18 18:33:24.854911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-18 18:33:24.858345: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 18:33:24.866818: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-18 18:33:25.505599: W tensorflow/compiler/tf2

In [2]:
ticker = "GLD"
window_days = 55
alpha = 0.5
star_lbdays=0
predict_days=5
data_label = MA4.data_label(ticker, window_days, alpha, predict_days, star_lbdays)
print(data_label)

GLD_55_0.5_5_0


In [3]:
batch_size = 32
epochs = 30
model_name = "CNN"
# model_name = "Dense"
model_label = MA4.model_label(data_label, model_name, epochs, batch_size)
print(model_label)
model = MA4.retreive_model(model_label)
if model is not None:
    model.summary()

GLD_55_0.5_5_0_CNN_30_32


I0000 00:00:1752888822.176220  455704 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-18 18:33:42.212376: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model GLD_55_0.5_5_0_CNN_30_32 loaded successfully.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,723,339 (273.60 MB)

 Trainable params: 23,907,779 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 47,815,560 (182.40 MB)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

In [ ]:
enddate = date(2025, 7, 3)
enddate = "2025-07-03"
startdate = date(2025, 1, 1)
startdate = "2025-01-01"
print(startdate, ' , ', enddate)

In [ ]:
fulldata = MA4.download_setup_data(ticker=ticker, predict_days=predict_days, window_days=window_days, alpha=alpha, lback=star_lbdays, 
                               start_date=startdate, end_date=enddate)

In [ ]:
validate_charts, validate_labels, finaldata = MA4.generate_chart(data=fulldata, ticker=ticker, predict_days=predict_days, 
                                                             window_days=window_days, datalabel_=data_label)

In [ ]:
def model_predict( model, X_test):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    return y_pred_classes

In [ ]:
def convert2Labels(y_pred_classes):
    label_names=[]
    for pclass in y_pred_classes:
        label_names.append(MA4.LABEL_L[pclass])
    print(len(label_names))
    print(label_names)
    return label_names

In [ ]:
validate_df = pd.DataFrame({"chart_path": validate_charts, "label": validate_labels})
# display(validate_df)
validate_fullp = os.path.join("validate_metadata", f"{model_label}_metadata.csv")
# print(validate_fullp)
os.makedirs(os.path.dirname(validate_fullp), exist_ok=True)
validate_df.to_csv(validate_fullp, index=False)

# display(fulldata.head(30))
df_filtered = finaldata.dropna(subset=['predict_date'])
# display(df_filtered)

validate_X, validate_Y = MA4.load_data(validate_fullp)
print(validate_X.shape, "  ,  ", validate_Y.shape)

y_pred_classes = model_predict(model, validate_X)

y_pred_labels = convert2Labels(y_pred_classes)

df_filtered['pred_Labels'] = y_pred_labels

start_d = df_filtered.index[0].strftime('%Y%m%d')
end_d = df_filtered.index[0].strftime('%Y%m%d')
result_fullp = os.path.join("validate_results", f"{model_label}_{start_d}_{end_d}.csv")
os.makedirs(os.path.dirname(result_fullp), exist_ok=True)
df_filtered.to_csv(result_fullp, index=False)

acc, prec, recall, f1 = MA4.cal_accuracy(ticker, validate_Y, y_pred_classes, model_name, model_label)
print(acc, prec, recall, f1)